# Write tiles to COG Experimentation

## Define Raster Tile Template

In [ ]:
"""tests ml_export.tile_generator.base"""

import os
import pytest
## Note, for mac osx compatability import something from shapely.geometry before importing fiona or geopandas
## https://github.com/Toblerity/Shapely/issues/553  * Import shapely before rasterio or fioana
from shapely import geometry
import mercantile
from rio_tiler import main
import numpy as np
from ml_export import tile_generator, tile_aggregator
import rasterio
from ml_export.tile_class_generator import TileClassDataset as TDC
from ml_export.ml_tools.mlbase import ml_tf_serving
%load_ext autoreload
%autoreload 2


In [ ]:
## Define Example Geometry

tile_obj = mercantile.tile(39.299515932798386, -6.080908028740757 , 12)

geom = geometry.box(-115.24, 36.1, -115.2, 36.2)
zoom_level = 12

large_tile_object_list = tile_generator.get_tile_list(geom, zoom=zoom_level)
print("{} z{} Tiles identified".format(len(large_tile_object_list), zoom_level))

In [ ]:
%%time
import os
access_token = os.getenv('MAPBOX_ACCESS_TOKEN')
print(access_token)
raster_tile_server_template = "https://a.tiles.mapbox.com/v4/mapbox.satellite/{z}/{x}/{y}.jpg?access_token="+access_token
file_name = "/Users/dlindenbaum/cosmiQGit/ml-export-tool/tests/fixtures/my-bucket/spacenet_test/vegas_test_inf_1.tiff"
api_location= 'http://localhost:8501/v1/models/looking_glass_export:predict'
api_location = 'http://gputest01.dev.labs.internal:8501/v1/models/looking_glass_export:predict'
large_tile_object = mercantile.tile(-115.24, 36.1986, 14) #las vegas
small_tile_zoom=18
super_res_tile_zoom = 18
mltf_obj = ml_tf_serving(api_location=api_location, output_num_channels=1)
tile_aggregator.build_cog_from_tiles_gen(file_name,
                             large_tile_object,
                             raster_tile_server_template,
                             desired_small_tile_zoom_level=small_tile_zoom,
                             desired_super_res_tile_zoom_level=super_res_tile_zoom,
                             cog=False,
                             indexes=None,
                             tile_size=256,
                             batch_size=5,
                             num_workers=4,
                             tile_dataset_class=TDC,
                                            detection_module=mltf_obj
                             )

In [ ]:
%%time
access_token = "pk.eyJ1IjoiZGF2aWRsaW5kZW5iYXVtIiwiYSI6ImNqb2QxbGQxbzB6amszd3J3bGV0ODJ4ODkifQ.yOQGasKWIF4sot6en8z8Eg"
raster_tile_server_template = "https://a.tiles.mapbox.com/v4/mapbox.satellite/{z}/{x}/{y}.jpg?access_token="+access_token
file_name = "/Users/dlindenbaum/cosmiQGit/ml-export-tool/tests/fixtures/my-bucket/spacenet_test/vegas_test.tiff"

large_tile_object = mercantile.tile(-115.24, 36.1986, 12) #las vegas
small_tile_zoom=18
super_res_tile_zoom = 18

tile_aggregator.build_cog_from_tiles_gen(file_name,
                             large_tile_object,
                             raster_tile_server_template,
                             desired_small_tile_zoom_level=small_tile_zoom,
                             desired_super_res_tile_zoom_level=super_res_tile_zoom,
                             cog=False,
                             indexes=None,
                             tile_size=256,
                             batch_size=4,
                             num_workers=4,
                             tile_dataset_class=TDC
                             )